In [1]:
import socket as sk
from threading import Thread, Lock
import time

In [13]:
def thr_accept():
#     global socket_dict
    while True:
        try: 
            with_client, addr = server.accept()
            _lock.acquire()
            socket_dict[addr] = with_client
            _lock.release()
            make_thr_recv(addr)
            broadcast('SYSTEM : Connection with ' + str(addr))
            broadcast('SYSTEM : # of Connection : ' + str(len(socket_dict)))
            print('SYSTEM : Connection with ' + str(addr))
            print('SYSTEM : # of Connection : ' + str(len(socket_dict)))
        except: 
            pass

In [11]:
def broadcast(data):
#     global socket_dict
    _lock.acquire()
    for with_client in socket_dict.values():
        try: with_client.send(data.encode('utf-8'))
        except: pass
    _lock.release()

In [4]:
def make_thr_recv(addr):
    t = Thread(target=thr_recv, args=(addr,))
    t.daemon = True
    t.start()

In [14]:
def thr_recv(addr):
#     global socket_dict
    sock = socket_dict[addr]
    while True:
        try: 
            data = sock.recv(1024).decode('utf-8')
            if data == '/quit':
                sk_close(addr)
                broadcast('SYSTEM : Client' + str(addr) + ' quit.')
                print('SYSTEM : Client' + str(addr) + ' quit.')
                break
            broadcast('Client' + str(addr) + ' : ' + data)
            print('Client' + str(addr) + ' : ' + data)
        except: 
            pass

In [6]:
def sk_close(addr):
#     global socket_dict
    _lock.acquire()
    socket_dict[addr].close()
    del socket_dict[addr]
    _lock.release()
    broadcast('SYSTEM : # of Connection : ' + str(len(socket_dict)))
    print('SYSTEM : # of Connection : ' + str(len(socket_dict)))

In [15]:
HOST = ''
PORT = 12321
socket_dict = {}
_lock = Lock()

server = sk.socket(sk.AF_INET, sk.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen(0)

if __name__ == '__main__':
    t = Thread(target=thr_accept, args=())
    t.daemon = True
    t.start()
    
    print('Run')
    
    while True: # to interrupt
        try:
            time.sleep(1)
        except:
            for with_client in socket_dict.values():
                with_client.close()
            server.close()
            print('Shutdown')
            break

Run
SYSTEM : Connection with ('127.0.0.1', 56392)
SYSTEM : # of Connection : 1
Client('127.0.0.1', 56392) : asdf
Shutdown
